# 'Real' System

We provide a semi-real system.

The system is as follow:

We want to compute an estimation of $\pi$.
One way to do this is to use Monte-Carlo simulations.

The idea of Monte-Carlo simulations is to execute **a lot of small and independant simulations** and compute the final result based on the results of the simulations.

In our case, each simulation we draw a random number $x$ in $[-1, 1]$, and then compute (in a very inefficient way) $\sqrt{1 - x^2}$.

The final result is the sum of each simulation, which is an approximation of $\int_{-1}^1 \sqrt{1-x^2}dx = \frac{\pi}{2}$

Our sensor is the `loadavg` of the machine. The `loadavg` is a metric representing the CPU utilization.

Our actuator is the number of threads excuting simulations in parallel (between 0 and 8).

> **Our control objective is to control the `loadavg` metric around a given value by adapting the number of threads executing simulations.**

You can run the system with the following `docker` command:

```sh
docker run --privileged -it -v $(pwd):/data registry.gitlab.inria.fr/control-for-computing/tutorial/system:v0.0 tuto-ctrl main.lua 1000000
```

The `main.lua` file is the file containing the controller code, and the last parameter is the total number of iterations to do.

<div class="alert alert-info" role="alert">
   Your tasks:
   <ol>
      <li>Play with the system: Try different constant inputs and see the output</li>
      <li>Assuming the underlying model is a first order model: Perform the identification</li>
      <li>Design a PI Controller for this systems</li>
      <li>Introduce pertubations: At some point in your experiment, run `yes` in another terminal to act as a disturbance, and see how your controller reacts.
    </ol> 
</div>

## Why `lua`?

Because it is a simple, small language that integrate with `C` easily!
Instead of giving you the source code and asking you to write `C` code to implement the controller, we can just write the controller in `lua` and pass the `lua` file as an argument of the `C` binary to be loaded.

You can find a `lua` cheat sheet [here](https://devhints.io/lua).

We define here a Proportional Controller:

```lua
-- main.lua file

-- Do not worry about this part
Controller = {}
Controller.__index = Controller

function Controller:new()
    -- Here we write the fields of the object.
    -- For a P controller we need a proportional gain (kp)
    -- and the reference value (ref)
    local ret = {kp = 0.5, ref = 2.0}
    setmetatable(ret, Controller)
    return ret
end

function Controller:ctrl(sensor)
    -- This is the main function of the controller.
    -- It gets the sensor value from the system,
    -- and must return the next input
    
    -- First we compute the control error
    err = self.ref - sensor
    -- Then we compute the next u
    u = self.kp * err
    -- We make sure the value of u makes sense for our system
    if u < 0 then
        u = 0
    end
    -- We print a line with the different metrics
    print("sensor: " .. sensor .. ", ref: " .. self.ref .. ", actuator: " .. u .. ", nb threads: " .. math.floor(u))
    
    -- We finally return the value for the next input
    return math.floor(u)
end

return Controller
```

[Back to menu](./00_Main.ipynb)